# Exercise 1: Agent Hello World

**Goal:** Create a simple AI agent with a custom weather tool using the OpenAI Agents SDK.

**Timebox:** 15 minutes

## What You'll Learn

- How to create an agent using the OpenAI Agents SDK
- How to define custom tool functions with `@function_tool`
- How to make real API calls (National Weather Service)
- How to run the agent with asyncio


> **💡 Tip:** If you get stuck, check the solution notebook in the `solutions/` directory! Each exercise has a corresponding complete solution that you can reference.
## Prerequisites

Before starting, ensure you have:

1. **OpenAI API Key:** Add your key to `.env` in the project root:
   ```
   OPENAI_API_KEY=sk-your-key-here
   ```

2. **Dependencies:** Already installed if you ran `make setup`
   - `openai-agents` - OpenAI Agents SDK
   - `httpx` - Async HTTP client
   - `python-dotenv` - Environment variable management

3. **Internet Connection:** Required for real National Weather Service API calls

## Key Concepts: OpenAI Agents SDK

### What is the OpenAI Agents SDK?

The **OpenAI Agents SDK** is a modern framework for building agentic AI applications. It provides:

- **Simplified agent creation** with the `Agent` class
- **Automatic tool schema generation** with `@function_tool` decorator
- **Built-in conversation management** via `Runner`
- **Async/await support** for efficient I/O operations

### Architecture Pattern 🏗️

```
User Query 👤
    ↓
Agent (OpenAI LLM) 🤖
    ↓
[Does user need tool?]
    ↓ Yes
Tool Function: get_weather_alerts() 🔧
    ↓
External API Call (NWS) 🌐
    ↓
Data returned to Agent 📊
    ↓
Agent generates response 💬
    ↓
Return to user ✅
```

This flow demonstrates how agents intelligently decide when to use tools to fulfill user requests!

### Tool Functions

Tool functions are Python functions that agents can call to perform actions:

- Decorated with `@function_tool` for automatic schema generation
- Can call external APIs, databases, or any Python code
- Return values are passed back to the LLM for final response generation
- The LLM decides **when** and **how** to use tools based on user queries

### Real API Integration

In this exercise, we use the **National Weather Service (NWS) API**:

- Provides free, real-time weather data for the United States
- No API key required (but User-Agent header recommended)
- Production-ready example of integrating external data sources
- Demonstrates how agents can access live information

### Why This Matters

This exercise teaches the **fundamental pattern** for all AI agents:
1. Define tools (functions the agent can use)
2. Create agent with instructions
3. Run agent with user query
4. Agent decides which tools to call
5. Tools return data
6. Agent generates final response

This same pattern scales from simple weather queries to complex multi-step workflows!

## Setup

Import required libraries and load environment variables.

Select **Python 3.11**** if prompted:

![Select Python 3.11 kernel](../../images/select-kernel.png)

In [1]:
%pip install --quiet openai-agents httpx python-dotenv nest-asyncio

import os
import asyncio
import httpx
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
import nest_asyncio

# Load environment variables
load_dotenv()

print("✅ Environment loaded")


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ Environment loaded


## Step 1: Define a Weather Tool

Create a `get_weather_alerts()` function that calls the National Weather Service API.

**TODO:** Implement the function to fetch real weather alerts.

**Hints:**
- API endpoint: `https://api.weather.gov/alerts/active/area/{state}`
- Use `async with httpx.AsyncClient() as client`
- Set headers: `{"User-Agent": "OpenAI-Agents-Workshop (educational)"}`
- Parse JSON response and extract `features` array
- Handle cases where there are no active alerts

In [2]:
@function_tool
async def get_weather_alerts(state: str) -> str:
    """
    Get current weather alerts for a US state from the National Weather Service.
    
    Args:
        state: Two-letter US state code (e.g., 'CA', 'NY', 'TX')
    
    Returns:
        String describing current weather alerts
    """
    print(f"🌐 Calling NWS API for state: {state}")

    try:
        # Call the National Weather Service API
        # TODO fill in the url: https://api.weather.gov/alerts/active/area/
        url = f"/{state.upper()}"
        headers = {"User-Agent": "OpenAI-Agents-Workshop (educational)"}

        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=headers, timeout=10.0)
            response.raise_for_status()

            data = response.json()
            features = data.get("features", [])

            if not features:
                print("✅ No active alerts")
                return f"No active weather alerts for {state.upper()}."

            # Extract alert information
            alerts = []
            for feature in features[:5]:  # Limit to first 5 alerts
                properties = feature.get("properties", {})
                event = properties.get("event", "Unknown")
                headline = properties.get("headline", "No headline")
                severity = properties.get("severity", "Unknown")
                alerts.append(f"- {event} ({severity}): {headline}")

            result = f"Active weather alerts for {state.upper()}:\n" + "\n".join(alerts)
            print(f"✅ Found {len(features)} alert(s)")
            return result

    except httpx.HTTPError as e:
        error_msg = f"Failed to fetch weather alerts: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg
    except Exception as e:
        error_msg = f"Unexpected error: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg


print("✅ Weather tool defined with @function_tool")
print("ℹ️  This tool makes real API calls to weather.gov")

✅ Weather tool defined with @function_tool
ℹ️  This tool makes real API calls to weather.gov


## Step 2: Create an Agent

Use the OpenAI Agents SDK to create an agent with instructions and tools.

**TODO:** Define the agent using the `Agent` class.

**Hints:**
- Give it a descriptive name (e.g., "Weather Agent")
- Write clear instructions explaining its purpose
- Pass your tool function in the `tools` list

In [ ]:
# TODO: Create an agent with the Agents SDK
# Hint: docs: https://openai.github.io/openai-agents-python/tools/#function-tools
# agent = Agent(
#     name="Weather Agent",
#     instructions="You help users get weather alert information for US states",
#TODO: you can pass in a tool or an apple, which will you choose?
#     tools=[]
# )

## Step 3: Verify OpenAI API Key

Ensure your OpenAI API key is configured.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is not set. Add it to your .env file.")

masked_key = f"{api_key[:4]}...{api_key[-4:]}" if len(api_key) >= 8 else "***"
print(f"✅ OPENAI_API_KEY detected: {masked_key}")

## Step 4: Run the Agent

Execute the agent with an example query.

**TODO:** Complete the `run_agent()` function to run the agent and display results.

In [ ]:
async def run_agent():
    query = "Are there any weather alerts for California?"
    print(f"Query: {query}\n")
    
    # TODO: Run the agent using Runner.run()
    # Hint: result = await Runner.run(agent, query)
    # TODO: Display the final output
    # Hint: print(f"\n🤖 Agent Response:\n{result.final_output}\n")
    
    print("⚠️  Implement the TODOs above!")
    print("   Check the solution notebook for reference")

# Jupyter-specific async handling (notebooks already have an event loop)
# In a regular Python script, you would use: asyncio.run(run_agent())
try:
    # Check if we're already in an async context (like Jupyter notebook)
    loop = asyncio.get_running_loop()
    # Allow nested async calls in the existing event loop
    nest_asyncio.apply()
    # Run our async function in the existing loop
    await run_agent()
except RuntimeError:
    # No event loop exists, so create a new one and run our function
    # This happens in regular Python scripts or non-async environments
    asyncio.run(run_agent())

Query: Are there any weather alerts for California?

⚠️  Implement the TODOs above!
   Check the solution notebook for reference


## Expected Output

When you complete the TODOs, you should see output similar to:

```
Query: Are there any weather alerts for California?

🌐 Calling NWS API for state: CA
✅ Found 3 alert(s)

🤖 Agent Response:
Currently there are 3 active weather alerts for California including:
- Wind Advisory (Moderate): Wind Advisory until midnight
- High Surf Advisory (Moderate): High Surf Advisory until 9 PM
- Freeze Warning (Moderate): Freeze Warning from 2 AM to 9 AM

ℹ️  This used a real API call to the National Weather Service!
```

**What's happening:**
1. Your agent receives the query about California
2. The LLM decides to call `get_weather_alerts("CA")`
3. Your function makes a real API call to weather.gov
4. The API returns current alert data
5. The LLM formats the data into a natural language response

## Troubleshooting

### Error: `OPENAI_API_KEY is not set`

**Fix:** Ensure `.env` file exists in project root with your API key:
```
OPENAI_API_KEY=sk-your-key-here
```

### Error: Connection timeout or network error

**Possible causes:**
- Check your internet connection
- NWS API may be temporarily unavailable (normal behavior)
- Try a different state code
- Verify the API endpoint URL is correct

### No active alerts returned

**This is normal!** Many states have no active weather alerts at any given time.

**To test with active alerts:**
- Try different states: CA, TX, FL, NY, etc.
- Check [weather.gov](https://www.weather.gov) to find states with current alerts
- Winter months typically have more alerts

### Agent not calling the tool

**Possible causes:**
- Check that `tools=[get_weather_alerts]` is passed to `Agent()`
- Verify the function is decorated with `@function_tool`
- Ensure your query mentions weather or alerts for a US state

## Next Steps

Once you've completed this exercise, you'll understand:

- ✅ How to create agents with the OpenAI Agents SDK
- ✅ How to define custom tools that call real APIs
- ✅ How the agent decides when to use tools
- ✅ How to run agents with async/await

**Ready for more?** Proceed to:

- **[Exercise 2: Temporal Hello World](../02_temporal_hello_world/exercise.ipynb)** - Learn Temporal workflows and activities
- **[Exercise 3: Durable Agent](../03_durable_agent/exercise.ipynb)** - Combine agents + Temporal for production durability!